## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)

# SQL Agent Scripts

- #1 List current eneabled / disabled jobs - 

> **Job information**
> 
> - List Jobs enabled, disabled by Name (Q1)
> - Job history specific job steps (Q3.1)
>  **Passwords**
- Update user passwords (Quer4)
- Add user to sysadmin - AD Connection (Query 5)
- Add use to server and sysadmin SQL user (Query 6)
- Add User to database role - data reader (query 7)

### **<u>Enabled jobs (Query 1) </u>**
> List Jobs enabled, disabled by Name 

In [ ]:
SELECT Name
    , enabled
FROM MSDB.dbo.sysjobs
ORDER BY Enabled DESC
    , Name


# Job History

### **<u>Job History (Q</u>uery <u>2)</u>**

> Details of SQL job history

In [9]:
SELECT      
      j.Name AS 'Job Name', 
    '"' + NULLIF(j.Description, 'No description available.') + '"' AS 'Description',
    SUSER_SNAME(j.owner_sid) AS 'Job Owner',  
     (SELECT COUNT(step_id) FROM msdb.dbo.sysjobsteps WHERE job_id = j.job_id) AS 'Number of Steps',
     (SELECT COUNT(step_id) FROM msdb.dbo.sysjobsteps WHERE job_id = j.job_id AND command LIKE '%xp_cmdshell%') AS 'has_xpcmdshell',
    'Frequency' = CASE s.freq_type
        WHEN 1 THEN 'Once'
        WHEN 4 THEN 'Daily'
        WHEN 0 THEN 'Once'
        ELSE cast('Every ' 
                + right(s.freq_subday_interval,2) 
                + ' '
                +     CASE(s.freq_subday_type)
                            WHEN 1 THEN 'Once'
                            WHEN 4 THEN 'Minutes'
                            WHEN 8 THEN 'Hours'
                        END as char(16))
    END,
    'Start Time'= CONVERT(DATETIME, RTRIM(NULLIF(js.next_run_date, 0)) + ' '
        + STUFF(STUFF(REPLACE(STR(RTRIM(js.next_run_time),6,0),
        ' ','0'),3,0,':'),6,0,':')),      
    'Max Duration' = STUFF(STUFF(REPLACE(STR(maxdur.run_duration,7,0),
        ' ','0'),4,0,':'),7,0,':'),
    'Last Run Duration' = STUFF(STUFF(REPLACE(STR(lastrun.run_duration,7,0),
        ' ','0'),4,0,':'),7,0,':'), --,
    'Last Start Date' = CONVERT(DATETIME, RTRIM(lastrun.run_date) + ' '
        + STUFF(STUFF(REPLACE(STR(RTRIM(lastrun.run_time),6,0),
        ' ','0'),3,0,':'),6,0,':')), 
    'Last Run Message' = lastrun.message,
    'Job Type' = CASE SUBSTRING(j.name,1,3)
        WHEN 'DMS' THEN 'DMS'
        WHEN 'GLO' THEN 'DMS'
        ELSE 'Maintenance'
      END,
     'Category' = CASE j.category_id
        WHEN 0 THEN 'Unclassified'
        WHEN 3 THEN 'Database Maintenance'
        WHEN 100 THEN 'DBA Tasks'
        ELSE 'unknown'
      END,
      j.date_modified, j.version_number, --mj 
 
     'Job Enabled' = CASE j.Enabled
        WHEN 1 THEN 'Yes'
        WHEN 0 THEN 'No'
      END
    
FROM msdb.dbo.sysjobs j
LEFT OUTER JOIN msdb.dbo.sysjobschedules js
    ON j.job_id = js.job_id
LEFT OUTER JOIN msdb.dbo.sysschedules s
    ON js.schedule_id = s.schedule_id 
LEFT OUTER JOIN (SELECT job_id, max(run_duration) AS run_duration
        FROM msdb.dbo.sysjobhistory
        GROUP BY job_id) maxdur
ON j.job_id = maxdur.job_id

LEFT OUTER JOIN
    (SELECT j1.job_id, j1.run_duration, j1.run_date, j1.run_time, j1.message
    FROM msdb.dbo.sysjobhistory j1
    WHERE instance_id = (SELECT MAX(instance_id) 
                         FROM msdb.dbo.sysjobhistory j2 
                         WHERE j2.job_id = j1.job_id)) lastrun
    ON j.job_id = lastrun.job_id
      --WHere (j.name like 'DBA - Main%'        or j.name like 'DMS -%'       or j.name like 'Global %' ) --   and j.[enabled] = 1   -- muz
ORDER BY [Job Name]


### **<u>Job history specific job (Q3)</u>**
> Details of SQL job history for specified job  
> Source [https://stevestedman.com/2016/03/tsql-script-display-agent-job-history/](https://stevestedman.com/2016/03/tsql-script-display-agent-job-history/)

### **<u>Job history specific job steps (Q3.1)</u>**
> Details of SQL job history steps for specified job

In [ ]:
--Get Times for each job steps for specified Job
DECLARE @JobName VARCHAR(50) ='VST__PreAllocateTransfers_Vector'

SELECT 
  sj.name JobName
, sjh.step_id
, ISNULL(sjs.step_name, 'Job Status') StepName
, msdb.dbo.agent_datetime(run_date, run_time) AS run_datetime
, STUFF(STUFF(RIGHT('00000' + CAST(run_duration AS VARCHAR(6)),6),3,0,':'),6,0,':') AS TimeTaken
, sjh.message
FROM msdb.dbo.sysjobs sj
  INNER JOIN msdb.dbo.sysjobhistory sjh ON sj.job_id = sjh.job_id
  LEFT OUTER JOIN msdb.dbo.sysjobsteps sjs ON sjh.job_id = sjs.job_id AND sjh.step_id = sjs.step_id 
  WHERE sj.name = @JobName
  AND sjh.step_id=6
ORDER BY  msdb.dbo.agent_datetime(run_date, run_time) DESC


In [3]:
--https://stevestedman.com/2016/03/tsql-script-display-agent-job-history/ - added in jobsteps
DECLARE @JobName VARCHAR(50) ='Automated Decommissioning'

;WITH jobListCTE as
(
SELECT j.name as job_name,js.step_id,js.step_name,
msdb.dbo.agent_datetime(run_date, run_time) AS run_datetime,
RIGHT('000000' + CONVERT(varchar(6), run_duration), 6) AS run_duration,
message
FROM msdb.dbo.sysjobhistory h
INNER JOIN msdb.dbo.sysjobs j ON h.job_id = j.job_id
INNER JOIN msdb.dbo.sysjobsteps js ON h.job_id = js.job_id
WHERE h.step_name = '(Job outcome)'
)
SELECT job_name as [JobStep],step_id,step_name,
run_datetime as [StartDateTime],
SUBSTRING(run_duration, 1, 2) + ':' +
SUBSTRING(run_duration, 3, 2) + ':' +
SUBSTRING(run_duration, 5, 2) as [Duration],
message
FROM jobListCTE
Where job_name =@JobName
ORDER BY run_datetime DESC, job_name;

## Job timings all Jobs (Q)

In [1]:
;WITH jobListCTE as
(
SELECT j.name as job_name,js.step_id,js.step_name,
msdb.dbo.agent_datetime(run_date, run_time) AS run_datetime,
RIGHT('000000' + CONVERT(varchar(6), run_duration), 6) AS run_duration,
message
FROM msdb.dbo.sysjobhistory h
INNER JOIN msdb.dbo.sysjobs j ON h.job_id = j.job_id
INNER JOIN msdb.dbo.sysjobsteps js ON h.job_id = js.job_id
)
SELECT job_name as [JobStep],step_id,step_name,
run_datetime as [StartDateTime],
SUBSTRING(run_duration, 1, 2) + ':' +
SUBSTRING(run_duration, 3, 2) + ':' +
SUBSTRING(run_duration, 5, 2) as [Duration],
message
FROM jobListCTE

ORDER BY run_datetime DESC, job_name;

## SQL Job Steps

In [ ]:
SELECT
    sj.name,
    sjs.step_id,
    sjs.step_name,
    sjs.command,
    sjs.subsystem,
    sjs.database_name
    ,sjs.on_success_action
FROM MSDB.dbo.sysjobsteps sjs
JOIN MSDB.dbo.sysjobs sj on sj.job_id=sjs.job_id
ORDER BY sj.name

## SQL Jobsteps formatted

In [5]:

WITH CTE_JobSteps
AS
(
SELECT sj.Name,sj.[description],sjs.step_name,sjs.step_id,sjs.command,sjs.subsystem,sjs.database_name, ROW_NUMBER() OVER (Partition by sj.Name  Order By sj.Name,sjs.step_id ) AS RowID, 1 AS Show
FROM msdb.dbo.sysjobs sj
INNER JOIN msdb.dbo.sysjobsteps sjs ON sjs.job_id =sj.job_id
WHERE sj.enabled =1
UNION ALL
SELECT sj.Name ,'' AS description,'' AS step_name,'' AS step_id,'' AS command,'' AS subsystem,'' AS database_name,MAX(step_id)+1 AS RowID,0 AS Show
FROM msdb.dbo.sysjobs sj
INNER JOIN msdb.dbo.sysjobsteps sjs ON sjs.job_id =sj.job_id
WHERE sj.enabled =1
GROUP BY sj.name
)
SELECT 
    CASE WHEN show =1 THEN  Name ELSE '' END AS Name
    ,CASE WHEN show =1 THEN  CAST(Step_id AS CHAR(4)) ELSE '' END AS Step_id
    ,step_name
    ,command
    ,subsystem
    ,database_name
FROM CTE_JobSteps c
ORDER BY c.Name,c.RowID

## Job Failures

In [ ]:
;WITH jobListCTE as
(
SELECT j.name as job_name,js.step_id,js.step_name,
msdb.dbo.agent_datetime(run_date, run_time) AS run_datetime,
RIGHT('000000' + CONVERT(varchar(6), run_duration), 6) AS run_duration,
message
FROM msdb.dbo.sysjobhistory h
INNER JOIN msdb.dbo.sysjobs j ON h.job_id = j.job_id
INNER JOIN msdb.dbo.sysjobsteps js ON h.job_id = js.job_id
WHERE run_status <> 1
)

SELECT job_name as [JobStep],step_id,step_name,
run_datetime as [StartDateTime],
SUBSTRING(run_duration, 1, 2) + ':' +
SUBSTRING(run_duration, 3, 2) + ':' +
SUBSTRING(run_duration, 5, 2) as [Duration],
message
FROM jobListCTE

ORDER BY run_datetime DESC, job_name;


## Job Schedules running at the sametime
source https://www.brentozar.com/blitz/agent-jobs-starting-simultaneously/

In [ ]:
SELECT j.name, j.description, a.start_execution_date
FROM msdb.dbo.sysjobs j
INNER JOIN msdb.dbo.sysjobactivity a ON j.job_id = a.job_id
WHERE a.start_execution_date > DATEADD(dd, -14, GETDATE())
AND j.enabled = 1
AND a.start_execution_date IN
(SELECT start_execution_date
FROM msdb.dbo.sysjobactivity
WHERE start_execution_date > DATEADD(dd, -14, GETDATE())
GROUP BY start_execution_date HAVING COUNT(*) > 1)
ORDER BY 3

# Running SQL Jobs
# ================================

## Start Job

## Start Job at Job Step

In [ ]:
USE msdb
GO
EXEC msdb.dbo.sp_start_job @job_name='Update_Search_Data', @step_name='Check Procedure Still Exists'



## Check Job history

In [ ]:

 select top 10 * from Search_Data

## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)